![](https://i.imgur.com/eBRPvWB.png)

# Practical PyTorch: Exploring Word Vectors with GloVe

When working with words, dealing with the huge but sparse domain of language can be challenging. Even for a small corpus, your neural network (or any type of model) needs to support many thousands of discrete inputs and outputs.

Besides the raw number words, the standard technique of representing words as one-hot vectors (e.g. "the" = `[0 0 0 1 0 0 0 0 ...]`) does not capture any information about relationships between words.

Word vectors address this problem by representing words in a multi-dimensional vector space. This can bring the dimensionality of the problem from hundreds-of-thousands to just hundreds. Plus, the vector space is able to capture semantic relationships between words in terms of distance and vector arithmetic.

![](https://i.imgur.com/y4hG1ak.png)

There are a few techniques for creating word vectors. The word2vec algorithm predicts words in a context (e.g. what is the most likely word to appear in "the cat ? the mouse"), while GloVe vectors are based on global counts across the corpus &mdash; see [How is GloVe different from word2vec?](https://www.quora.com/How-is-GloVe-different-from-word2vec) on Quora for some better explanations.

In my opinion the best feature of GloVe is that multiple sets of pre-trained vectors are easily [available for download](https://nlp.stanford.edu/projects/glove/), so that's what we'll use here.

## Recommended reading

* https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/
* https://blog.acolyer.org/2016/04/22/glove-global-vectors-for-word-representation/
* https://levyomer.wordpress.com/2014/04/25/linguistic-regularities-in-sparse-and-explicit-word-representations/

## Installing torchtext

The torchtext package is not currently on the PIP or Conda package managers, but it's easy to install manually:

```
git clone https://github.com/pytorch/text pytorch-text
cd pytorch-text
python setup.py install
```

## Loading word vectors

Torchtext includes functions to download GloVe (and other) embeddings

In [1]:
import torch
import torchtext.vocab as vocab

In [2]:
glove = vocab.GloVe(name='6B', dim=100)

print('Loaded {} words'.format(len(glove.itos)))

Loaded 400000 words


The returned `GloVe` object includes attributes:
- `stoi` _string-to-index_ returns a dictionary of words to indexes
- `itos` _index-to-string_ returns an array of words by index
- `vectors` returns the actual vectors. To get a word vector get the index to get the vector:

In [3]:
def get_word(word):
    return glove.vectors[glove.stoi[word]]

## Finding closest vectors

Going from word &rarr; vector is easy enough, but to go from vector &rarr; word takes more work. Here I'm (naively) calculating the distance for each word in the vocabulary, and sorting based on that distance:

Anyone with a suggestion for optimizing this, please let me know!

In [4]:
def closest(vec, n=10):
    """
    Find the closest words for a given vector
    """
    all_dists = [(w, torch.dist(vec, get_word(w))) for w in glove.itos]
    return sorted(all_dists, key=lambda t: t[1])[:n]

This will return a list of `(word, distance)` tuple pairs. Here's a helper function to print that list:

In [5]:
def print_tuples(tuples):
    for tuple in tuples:
        print('(%.4f) %s' % (tuple[1], tuple[0]))

Now using a known word vector we can see which other vectors are closest:

In [6]:
print_tuples(closest(get_word('google')))

(0.0000) google
(3.0772) yahoo
(3.8836) microsoft
(4.1048) web
(4.1082) aol
(4.1165) facebook
(4.3917) ebay
(4.4122) msn
(4.4540) internet
(4.4651) netscape


## Word analogies with vector arithmetic

The most interesting feature of a well-trained word vector space is that certain semantic relationships (beyond just close-ness of words) can be captured with regular vector arithmetic. 

![](https://i.imgur.com/d0KuM5x.png)

(image borrowed from [a slide from Omer Levy and Yoav Goldberg](https://levyomer.wordpress.com/2014/04/25/linguistic-regularities-in-sparse-and-explicit-word-representations/))

In [6]:
# In the form w1 : w2 :: w3 : ?
def analogy(w1, w2, w3, n=5, filter_given=True):
    print('\n[%s : %s :: %s : ?]' % (w1, w2, w3))
   
    # w2 - w1 + w3 = w4
    closest_words = closest(get_word(w2) - get_word(w1) + get_word(w3))
    
    # Optionally filter out given words
    if filter_given:
        closest_words = [t for t in closest_words if t[0] not in [w1, w2, w3]]
        
    print_tuples(closest_words[:n])

The classic example:

In [7]:
analogy('king', 'man', 'queen')


[king : man :: queen : ?]
(4.0811) woman
(4.6916) girl
(5.2703) she
(5.2788) teenager
(5.3084) boy


Now let's explore the word space and see what stereotypes we can uncover:

In [8]:
analogy('man', 'actor', 'woman')
analogy('cat', 'kitten', 'dog')
analogy('dog', 'puppy', 'cat')
analogy('russia', 'moscow', 'france')
analogy('obama', 'president', 'trump')
analogy('rich', 'mansion', 'poor')
analogy('elvis', 'rock', 'eminem')
analogy('paper', 'newspaper', 'screen')
analogy('monet', 'paint', 'michelangelo')
analogy('beer', 'barley', 'wine')
analogy('earth', 'moon', 'sun') # Interesting failure mode
analogy('house', 'roof', 'castle')
analogy('building', 'architect', 'software')
analogy('boston', 'bruins', 'phoenix')
analogy('good', 'heaven', 'bad')
analogy('jordan', 'basketball', 'woods')


[man : actor :: woman : ?]
(2.8133) actress
(5.0039) comedian
(5.1399) actresses
(5.2773) starred
(5.3085) screenwriter

[cat : kitten :: dog : ?]
(3.8146) puppy
(4.2944) rottweiler
(4.5888) puppies
(4.6086) pooch
(4.6520) pug

[dog : puppy :: cat : ?]
(3.8146) kitten
(4.0255) puppies
(4.1575) kittens
(4.1882) pterodactyl
(4.1945) scaredy

[russia : moscow :: france : ?]
(3.2697) paris
(4.6857) french
(4.7085) lyon
(4.9087) strasbourg
(5.0362) marseille

[obama : president :: trump : ?]
(6.4302) executive
(6.5149) founder
(6.6997) ceo
(6.7524) hilton
(6.7729) walt

[rich : mansion :: poor : ?]
(5.8262) residence
(5.9444) riverside
(6.0283) hillside
(6.0328) abandoned
(6.0681) bungalow

[elvis : rock :: eminem : ?]
(5.6597) rap
(6.2057) rappers
(6.2161) rapper
(6.2444) punk
(6.2690) hop

[paper : newspaper :: screen : ?]
(4.7810) tv
(5.1049) television
(5.3818) cinema
(5.5524) feature
(5.5646) shows

[monet : paint :: michelangelo : ?]
(6.0782) plaster
(6.3768) mold
(6.3922) tile
(6.58